# This notebook is used to store variations of functions, in case other solutions are required

## Previous network's update and removal of duplicates

Previously the network was updated and then duplicates were removed. Function network_intersections_update was updated to avoid the creation of duplicates.

In [ ]:
def network_intersections_update(current_ntw_nodes, current_ntw_edges, intersection_nodes, projected_crs, function_logs=False):

    """ This function takes points with osmid located over existing edges (intersection_nodes) and updates
        a network. The intersection_nodes become new nodes and each intersected edge get split 
        into two separate edges with new 'u', 'v' and 'key' data.
    
	Args:
		current_ntw_nodes (geopandas.GeoDataFrame): GeoDataFrame containing the nodes from the network to update.
                                                    Requires a unique identifier 'osmid'.
        current_ntw_edges (geopandas.GeoDataFrame): GeoDataFrame containing the edges from the network to update.
                                                    Requires the unique identifiers 'u ,'v' and 'key'.
        intersection_nodes (geopandas.GeoDataFrame): GeoDataFrame containing the points in where each split is performed.
                                                    Requires points with 'osmid', and the edge to split ('u','v' and 'key').
        projected_crs (str, optional): String containing projected crs to be used depending on area of interest. 
                                        Defaults to "EPSG:6372".
        function_logs (bool,optional): Boolean that (if True) prints logs during the functions execution. Defaults to False.

                                                 
	Returns:
        updated_ntw_nodes (geopandas.GeoDataFrame): GeoDataFrame containing the updated nodes for the network.
        updated_ntw_edges (geopandas.GeoDataFrame): GeoDataFrame containing the updated edges for the network. 
        
	""" 
    print(f"Updating network...")
    
    # ------------------- INPUT USED - READ AND FILTER EDGES
    current_ntw_nodes = current_ntw_nodes.copy()
    current_ntw_nodes = current_ntw_nodes.to_crs(projected_crs)
    # Set an identifier to make it easier to locate nodes that resulted from an intersection between networks
    current_ntw_nodes['intersecting'] = 0

    current_ntw_edges = current_ntw_edges.copy()
    current_ntw_edges = current_ntw_edges.to_crs(projected_crs)
    # Set an identifier to make it easier to locate edges that were split
    current_ntw_edges['intersecting'] = 0
    # ------------------- INPUT USED - READ AND FILTER EDGES

    # Iterate over each intersection between both networks (intersection_nodes)
    for idx, node in intersection_nodes.iterrows():
        
        # 3.1 --------------- Split the current_ntw intersected edge using the intersection_node as clipping_point. 
        # ------------------- This split (Using function edge_clipping()) creates two separate edges:
        # ------------------- The first edge will be related to the starting_point_gdf (We'll set intersected edge 'u')
        # ------------------- The second edge will be related to the opposite side (Will be intersected edge 'v')
    
        # Current intersection_node's data
        intersection_node_osmid = node['osmid']
        intersected_u = node['u']
        intersected_v = node['v']
        intersected_key = node['key']
        intersected_retain_how = node['retain_how']        

        if function_logs:
            print(f"network_intersections_update(): Iterating over intersection node osmid {intersection_node_osmid}.") #Debugging check
            print(f"network_intersections_update(): Intersected edge with u {intersected_u} type {type(intersected_u)}.") #Debugging check
            print(f"network_intersections_update(): Intersected edge with v {intersected_v} type {type(intersected_v)}.") #Debugging check
            print(f"network_intersections_update(): Intersected edge with key {intersected_key} type {type(intersected_key)}.") #Debugging check
            
        # Extract current intersection node as a gdf (Becomes clipping_point_gdf in function edge_clipping)
        # (In most cases, osmid is the only value needed to identify the intersection node. In a very specific case where two edges from current_ntw_edges cross at the very same point
        #  where an edge from the other network created an intersection, 'u', 'v' and 'key' would also be needed)
        intersection_node_idx = (intersection_nodes.u==intersected_u)&(intersection_nodes.v==intersected_v)&(intersection_nodes.key==intersected_key)&(intersection_nodes.osmid==intersection_node_osmid)
        intersection_node = intersection_nodes.loc[intersection_node_idx].copy()
        intersection_node.reset_index(inplace=True,drop=True)
        
        # Extract current_ntw intersected edge (Becomes edge_gdf in function edge_clipping)
        try:
            # Try loading the edge registered as intersected in the intersection_nodes gdf.
            # ('intersected_u', 'intersected_v' and 'intersected_key')
            intersected_edge = current_ntw_edges.loc[(current_ntw_edges['u'] == intersected_u) & 
                                                     (current_ntw_edges['v'] == intersected_v) &
                                                     (current_ntw_edges['key'] == intersected_key)].copy()
            intersected_edge.reset_index(inplace=True,drop=True)
            # If it has len=0, it failed.
            if len(intersected_edge) == 0:
                fail_try
                
        except:
            # If it doesn't work, it means that the edge no longer exists (deleted in following steps in this function)
            # This happens because that edge had another intersection along its lenght and 
            # that original unsplit edge was split and deleted.
            # Now a new already split edge lies underneath the current intersection_node.
            # --> Find that split edge's data
            if function_logs:
                print(f"""network_intersections_update(): Searching for already split edge originating from edge with u {intersected_u}, v {intersected_v} and key {intersected_key}.""")
            # Create a VERY SMALL buffer around the intersection_node
            intersection_node_buffer = intersection_node.buffer(1e-9)
            intersection_node_buffer = gpd.GeoDataFrame(geometry=intersection_node_buffer)
            # Find and rewrite the data of the split edge underneath the intersection_node
            edge_data = intersection_node_buffer.sjoin(current_ntw_edges)
            if len(edge_data) == 1:
                intersected_u = edge_data.u.unique()[0]
                intersected_v = edge_data.v.unique()[0]
                intersected_key = edge_data.key.unique()[0]
            else:
                # If this happens, it means that there are two or more intersection_nodes located exactly
                # in this point over the intersected_edge. The first has already split the edge, the next is attempting.
                # Since the edge doesn't need to be split again, skip this intersection_node.
                continue
            
            # Retrieve intersected edge
            intersected_edge = current_ntw_edges.loc[(current_ntw_edges['u'] == intersected_u) & 
                                                     (current_ntw_edges['v'] == intersected_v) &
                                                     (current_ntw_edges['key'] == intersected_key)].copy()
            intersected_edge.reset_index(inplace=True,drop=True)
        
        # Extract current_ntw intersected edge's u node 
        # (Always becomes starting_point_gdf in function edge_clipping when using 'return_all')
        u_node = current_ntw_nodes.loc[(current_ntw_nodes['osmid'] == intersected_u)].copy()
        u_node.reset_index(inplace=True,drop=True)

        # Extract current_ntw intersected edge's v node 
        v_node = current_ntw_nodes.loc[(current_ntw_nodes['osmid'] == intersected_v)].copy()
        v_node.reset_index(inplace=True,drop=True)
    
        # Apply edge_clipping function and assign the corresponding 'u', 'v' or 'key' data.
        if intersected_retain_how == 'both':
            # Clip edge
            split_edge_gdf = edge_clipping(starting_point_gdf = u_node,
                                           edge_gdf = intersected_edge,
                                           clipping_point_gdf = intersection_node,
                                           projected_crs = projected_crs,
                                           return_all = True,
                                           function_logs = function_logs)
            # Assign data
            # When return_all=True in function edge_clipping, 
            # assigns 'starting' to the edge related to the starting_point_gdf
            # and 'ending' to edge on the opposite side.
            u_idx = split_edge_gdf.relation=='starting'
            split_edge_gdf.loc[u_idx,'u'] = intersected_u # We assigned 'u' as starting_point_gdf
            split_edge_gdf.loc[u_idx,'v'] = intersection_node_osmid # Intersection
            split_edge_gdf.loc[u_idx,'key'] = 0 #Since this 'u' and 'v' relation is new, key=0
        
            v_idx = split_edge_gdf.relation=='ending'
            split_edge_gdf.loc[v_idx,'u'] = intersection_node_osmid # Intersection
            split_edge_gdf.loc[v_idx,'v'] = intersected_v # Opposite side
            split_edge_gdf.loc[v_idx,'key'] = 0 #Since this 'u' and 'v' relation is new, key=0
        
        elif intersected_retain_how == 'u':
            # Clip edge
            split_edge_gdf = edge_clipping(starting_point_gdf = u_node,
                                           edge_gdf = intersected_edge,
                                           clipping_point_gdf = intersection_node,
                                           projected_crs = projected_crs,
                                           return_all = False)
            # Assign data
            split_edge_gdf.loc[0,'u'] = intersected_u
            split_edge_gdf.loc[0,'v'] = intersection_node_osmid # Intersection
            split_edge_gdf.loc[0,'key'] = 0 #Since this 'u' and 'v' relation is new, key=0
            
        elif intersected_retain_how == 'v':
            # Clip edge
            split_edge_gdf = edge_clipping(starting_point_gdf = v_node,
                                           edge_gdf = intersected_edge,
                                           clipping_point_gdf = intersection_node,
                                           projected_crs = projected_crs,
                                           return_all = False)
            # Assign data
            split_edge_gdf.loc[0,'u'] = intersection_node_osmid # Intersection
            split_edge_gdf.loc[0,'v'] = intersected_v
            split_edge_gdf.loc[0,'key'] = 0 #Since this 'u' and 'v' relation is new, key=0
            
        else:
            print(f"Error splitting edge with u {intersected_u}, v {intersected_v} and key {intersected_key}.")
            print("Make sure to include in gdf intersection_nodes column 'retain_how' with either 'u','v' or 'both'.")
            intended_crash    
    
        # 3.2 --------------- Register changes on current_ntw
        # Set an identifier to make it easier to locate nodes that resulted from an intersection between networks
        intersection_node['intersecting'] = 1
        # Prepare node for concatenation
        intersection_node = intersection_node[['osmid','intersecting','geometry']]
        # Add new node
        current_ntw_nodes = pd.concat([current_ntw_nodes,intersection_node])
        # Reset index
        current_ntw_nodes.reset_index(inplace=True,drop=True)
    
        # Keep all edges except the edge that was split
        # (Must remove to avoid duplicating edge's geometries)
        current_ntw_edges = current_ntw_edges.loc[~((current_ntw_edges['u'] == intersected_u) &
                                                    (current_ntw_edges['v'] == intersected_v) &
                                                    (current_ntw_edges['key'] == intersected_key))].copy()
        # Prepare edges for concatenation
        split_edge_gdf = split_edge_gdf[['u','v','key','geometry']]
        # Set an identifier to make it easier to locate edges that were split
        split_edge_gdf['intersecting'] = 1
        # Add new edge
        current_ntw_edges = pd.concat([current_ntw_edges,split_edge_gdf])
        # Reset index
        current_ntw_edges.reset_index(inplace=True,drop=True)

    print(f"Updated network. Formating output.")
    updated_ntw_nodes = current_ntw_nodes[['osmid','intersecting','geometry']].copy()
    # Set unique identifiers to int
    updated_ntw_nodes['osmid'] = updated_ntw_nodes['osmid'].astype('int')
    del current_ntw_nodes
    updated_ntw_edges = current_ntw_edges[['u','v','key','intersecting','geometry']].copy()
    # Set unique identifiers to int
    updated_ntw_edges['u'] = updated_ntw_edges['u'].astype('int')
    updated_ntw_edges['v'] = updated_ntw_edges['v'].astype('int')
    updated_ntw_edges['key'] = updated_ntw_edges['key'].astype('int')
    del current_ntw_edges
    
    # After iterating over both networks, return result
    return updated_ntw_nodes, updated_ntw_edges

In [ ]:
def drop_intersection_network_duplicates(nodes_gdf,edges_gdf):

    """ This function was created as a complement to function network_intersections_update().
        Whenever three or more lines intersect exactly at the same point, function network_intersections_update() creates duplicated nodes and edges.
        Those duplicates cannot be easly dropped since lines can be mirrored. Example:
        Line 1: u=1, v=2, key=0, geom=((1,1),(2,1))
        Line 2: u=2, v=1, key=0, geom=((2,1),(1,1))
        
        This function solves for those duplicated network geometries.
    
	Args:
		nodes_gdf (geopandas.GeoDataFrame): GeoDataFrame containing the nodes from the recently updated network.
                                                    Requires a unique identifier 'osmid'.
        edges_gdf (geopandas.GeoDataFrame): GeoDataFrame containing the edges from the recently updated network.
                                                    Requires the unique identifiers 'u ,'v' and 'key'.

	Returns:
        nodes_gdf (geopandas.GeoDataFrame): GeoDataFrame containing the updated nodes without duplicates.
        edges_gdf (geopandas.GeoDataFrame): GeoDataFrame containing the updated edges without duplicates.
        
	""" 

    # Copy input to avoid rewritting
    nodes_gdf = nodes_gdf.copy()
    edges_gdf = edges_gdf.copy()

    # 1.0 --------------- Dropping duplicates on nodes
    print(f"Dropping duplicates on updated nodes and edges.")
    # Dropping duplicates on nodes by using osmid and geometry.
    current_len = len(nodes_gdf)
    nodes_gdf.drop_duplicates(subset=['osmid','geometry'],inplace=True)
    updated_len = len(nodes_gdf)
    print(f"Dropped {current_len-updated_len} nodes that had the same osmid and geometry.")

    # 2.0 --------------- Dropping duplicates on edges
    
    # 2.1 --------------- Identify potential duplicates by comparing edge_id in a regular and inverted order
    # Unique edge id with regular order ('u','v','key')
    edges_gdf = create_unique_edge_id(edges_gdf)
    edges_gdf.rename(columns={'edge_id':'edge_id_1'},inplace=True)
    current_len = len(edges_gdf)
    edges_gdf.drop_duplicates(subset=['edge_id_1'],inplace=True)
    updated_len = len(edges_gdf)
    print(f"Dropped {current_len-updated_len} edges that had the same edge_id.")
    
    # Unique edge id with inverted order ('v','u','key')
    edges_gdf = create_unique_edge_id(edges_gdf,order='vukey')
    edges_gdf.rename(columns={'edge_id':'edge_id_2'},inplace=True)
    dup_inverted_lst = []
    
    # Identify edges where edge_id is in both regular and inverted order
    for edge_id in list(edges_gdf.edge_id_1.unique()):
        if edge_id in list(edges_gdf.edge_id_2.unique()):
            dup_inverted_lst.append(edge_id)

    # 2.2 --------------- Verify potential duplicates and register one of them to be dropped
    # Verify those edges are duplicated
    confirmed_dup_edge_lst = []
    already_dropped_dict = {}
    for edge_id in dup_inverted_lst:
        regular_edge = edges_gdf.loc[edges_gdf.edge_id_1==edge_id]
        inverted_edge = edges_gdf.loc[edges_gdf.edge_id_2==edge_id]
        
        # Discard different roads by length
        # (Two roads may share start and end point if they each form half a circle)
        regular_edge_length = regular_edge.length.unique()[0]
        inverted_edge_length = inverted_edge.length.unique()[0]
        if regular_edge_length != inverted_edge_length:
            continue
            
        # Discard loop roads 
        # (One road may have the same start-end and end-start if it is a loop. Also, it would have the same length)
        regular_edge_index = regular_edge.index[0]
        inverted_edge_index = inverted_edge.index[0]
        if regular_edge_index == inverted_edge_index:
            continue
            
        # If reached here, it is duplicated. Check if the mirror relation was already registered
        value_1 = regular_edge.u.unique()[0]
        value_2 = regular_edge.v.unique()[0]
        # If value_1 is already registered in the dictionary and it contains value_2, continue.
        if (value_1 in already_dropped_dict.keys()) and (value_2 in already_dropped_dict[value_1]):
            print(f"Relation {value_1}<-->{value_2} already registered.")
            continue
        # If value_2 is already registered in the dictionary and it contains value_1, continue.
        elif (value_2 in already_dropped_dict.keys()) and (value_1 in already_dropped_dict[value_2]):
            print(f"Relation {value_1}<-->{value_2} already registered.")
            continue
        
        # Else, it has not been registered
        else:
            # Confirm that this edge_id will be dropped
            confirmed_dup_edge_lst.append(edge_id)
            # Save the relation that's being dropped   
            if value_1 not in already_dropped_dict.keys():
                already_dropped_dict[value_1] = list([value_2])
                print(f"Saved relationship {value_1}<-->{value_2} to be dropped.")
            else:
                already_dropped_dict[value_1] = already_dropped_dict[value_1].append(value_2)
                print(f"Saved relationship {value_1}<-->{value_2} to be dropped.")
            
    # 2.3 --------------- Drop confirmed relations
    current_len = len(edges_gdf)
    edges_gdf = edges_gdf.loc[~edges_gdf.edge_id_2.isin(confirmed_dup_edge_lst)]
    updated_len = len(edges_gdf)
    print(f"Dropped {current_len-updated_len} edges that had the same edge_id but one was inverted.")
    # Drop columns used for dropping duplicates inside drop_intersection_network_duplicates()
    edges_gdf.drop(columns=['edge_id_1','edge_id_2'],inplace=True)

    return nodes_gdf,edges_gdf